In [ ]:
import requests
import seaborn as sns
import json
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd
import operator

In [ ]:
def seconds_to_mhs(sec: int) -> str:
    return f"{sec // 3600:02.0f}:{(sec % 3600) // 60:02.0f}:{(sec % 3600) % 60:02.0f}"

def find(path, dict, sep="/"):
    keys = path.split(sep)
    rv = dict
    for key in keys:
        rv = rv[key]
    return rv

In [ ]:
# Adventure Walk 2024 - 50 km
# base_url = 'https://my1.raceresult.com/266144/RRPublish/data/list?key=cbb56f3829f252176e762b342c83e242&listname=Ergebnislisten%7CErgebnisliste%20-%2050k&page=results&contest=0&r=all&l=0'

# Adventure Walk 2024 - 30 km
# base_url = 'https://my1.raceresult.com/266144/RRPublish/data/list?key=cbb56f3829f252176e762b342c83e242&listname=Ergebnislisten%7CErgebnisliste%20-%2025k&page=results&contest=0&r=all&l=0'

# Dresden Marathon 2024 - "Allgemein - Ergebnisliste Männer/Frauen"
# base_url = "https://my2.raceresult.com/270281/RRPublish/data/list?key=aaee10feb460178330e3e6495129af10&listname=Ergebnislisten%7CErgebnisliste%20M%C3%A4nner%2FFrauen&page=results&contest=0&r=all&l=0"

# SachsenTrail 2024 - "UltraTrail (75,5 km & 2120 Höhenmeter)"
base_url = "https://my1.raceresult.com/250768/RRPublish/data/list?key=adf2c8d4f3db7f4f7113e197b3bd20de&listname=Ergebnislisten%7CErgebnisliste&page=results&contest=1"

# REWE Team Challenge 2024 - "Einzelwertung Männer"
# base_url = "https://my4.raceresult.com/290895/RRPublish/data/list?key=6630ea4ed4b803531ec88084f95a5eff&listname=Ergebnislisten%7CInternet-einzel%20-%20M%C3%A4nner&page=results&contest=1"

response = requests.get(f"{base_url}")

In [ ]:
data = json.loads(response.text)

In [ ]:
def print_filters(group_filter_dict, max_level, current_level):
    if current_level == 0:
        return
    for key in group_filter_dict.keys():
        print((max_level - current_level) * "  ", key)
        print_filters(group_filter_dict[key], max_level, current_level-1)

filter_level_count = len(data['groupFilters'])
print("There are", filter_level_count, "levels of filter.")
print("available competitions and potential sub-groups:")
print_filters(data['data'], filter_level_count, filter_level_count)

In [ ]:
# for REWE Team Challenge
columns = ["id"] + [field["Label"].lower() for field in data["list"]["Fields"]]
print("Columns:", columns)

clss = "#1_Männer"

data_path = clss
my_name = "Stanley Förster"

In [ ]:
# for Sachsen Trail 2024
columns = ["id"] + [field["Label"].lower() for field in data["list"]["Fields"][:-1]]
print("Columns:", columns)

competition = "#1_UltraTrail (75,5 km & 2120 Höhenmeter)"
clss = "#2_Männer"
# clss = "#1_Frauen

data_path = f"{competition}/{clss}"
my_name = None

In [ ]:
# for Dresden Marathon 2024
columns = ["id"] + [field["Label"].lower() for field in data["list"]["Fields"][:-2]]

# competition = "#1_{DE:AOK-Viertelmarathon (10,55 km)|EN:AOK Quarter marathon (10.55 k)|CZ:čtvrtmaraton (10,55 km)}"
# clss = "#1_Männer"
# clss = "#2_Frauen"

# competition = "#2_{DE:Halbmarathon|EN:half marathon|CZ:půlmaraton}"
# clss = "#3_Männer"
# clss = "#4_Frauen"

competition = "#3_{DE:Marathon|EN:marathon|CZ:maratón}"
clss = "#5_Männer"
# clss = "#6_Frauen"

# competition = "#4_{DE:Sparkassen Zehntelmarathon (4,2 km)|EN:Sparkassen 1/10 marathon (4,2km)|CZ:desátýmaratón (4,2 km)}"
# clss = "#7_Männer"
# clss = "#8_Frauen"

data_path = f"{competition}/{clss}"
my_name = "FÖRSTER, Stanley"

In [ ]:
col_time = "zeit"
df = (
    pd.concat(
        [
            pd.DataFrame(values[: len(columns)], index=columns).T
            for values in find(data_path, data["data"])
        ]
    )
    .reset_index(drop=True)
    .assign(
        time=lambda df: pd.to_timedelta(df[col_time]),
        seconds=lambda df: df["time"].dt.total_seconds(),
    )
)

In [ ]:
# for Adventure Walk 2024 - here, the "classes" are groups of people by first letter of the name ...
idx_starter_id = 0
idx_name = 1
idx_time = 2

my_name = "Förster, Stanley"
df = pd.concat([
    pd.DataFrame(values, columns=['id', 'name', 'time', 'misc']) for _, values in list(data['data'].values())[0].items()
]).reset_index(drop=True).assign(time=lambda df: pd.to_timedelta(df['time']), seconds=lambda df: df['time'].dt.total_seconds())

In [ ]:
# TODO - allow adding arbitrary names and improve markings
# TODO - visualize quartiles via background shading
with plt.style.context("bmh"):
    sns.histplot(
        data=df,
        x='seconds',
    )

    my_record = {
        "my": records.iloc[0]['seconds'],  # TODO - better go with bib number instead of name
    } if len(records := df.query("name == @my_name")) > 0 else {}

    palette = sns.color_palette("bright")
    for i, (prefix, value) in enumerate(dict({
        "min": df['seconds'].min(),
        "median": df['seconds'].median(),
        "max": df['seconds'].max(),
    } | my_record).items()):
        plt.axvline(
            value,
            label=f"{prefix}: {seconds_to_mhs(value)}",
            color=palette[1+i],
        )
    ax = plt.gca()
    ax.tick_params(axis='x', labelrotation=30)
    ax.xaxis.set_major_formatter(FuncFormatter(func=lambda x, pos: seconds_to_mhs(x)))
    plt.xlabel("Time")
    plt.ylabel(f"Participants (total = {len(df)})")
    plt.title(f"{data['list']['HeadLine1']}")

    plt.legend()
    plt.show()